In [ ]:
# ruff: noqa: F405

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from wedowind_example import *  # noqa: F403

In [3]:
download_wdw_data_from_zenodo()

File C:\Users\snaylor\Documents\GitHub\wind-up\cache\wedowind_example_data\Turbine_Upgrade_Dataset.zip already exists. Skipping download.
File C:\Users\snaylor\Documents\GitHub\wind-up\cache\wedowind_example_data\Inland_Offshore_Wind_Farm_Dataset1.zip already exists. Skipping download.


In [4]:
assumed_rated_power_kw = 1500
rotor_diameter_m = 80
cutout_ws_mps = 20
scada_file_name = "Turbine Upgrade Dataset(Pitch Angle Pair).csv"  # or Turbine Upgrade Dataset(VG Pair).csv

In [5]:
logger.info("Preprocessing turbine SCADA data")
scada_df = WDWScadaUnpacker(scada_file_name=scada_file_name).unpack(rated_power_kw=assumed_rated_power_kw)
scada_df.head()

Preprocessing turbine SCADA data


,upgrade status,V,D,rho,S,I,normalized_power,TurbineName,ActivePowerMean,WindSpeedMean,YawAngleMean,PitchAngleMean,GenRpmMean,ShutdownDuration
TimeStamp_StartFormat,,,,,,,,,,,,,,
2010-07-30 22:40:00+00:00,0,7.96,138.9,1.140224,0.266512,0.090452,0.393152,Test,589.727273,7.96,138.9,0,1000,0
2010-07-30 22:50:00+00:00,0,8.19,140.6,1.140522,0.286167,0.083028,0.457455,Test,686.181817,8.19,140.6,0,1000,0
2010-07-30 23:00:00+00:00,0,7.20,139.3,1.140771,0.339321,0.098611,0.382121,Test,573.181818,7.20,139.3,0,1000,0
2010-07-30 23:10:00+00:00,0,6.81,137.4,1.141186,0.375815,0.101322,0.282182,Test,423.272727,6.81,137.4,0,1000,0
2010-07-30 23:20:00+00:00,0,5.09,137.5,1.141464,0.303472,0.165029,0.127212,Test,190.818182,5.09,137.5,0,1000,0


In [6]:
metadata_df = make_wdw_metadata_df()
metadata_df.head()

,Name,Latitude,Longitude,TimeZone,TimeSpanMinutes,TimeFormat
0,WT1,40.036394,-89.052141,UTC,10,Start
1,WT2,40.039089,-89.032205,UTC,10,Start
2,WT3,39.954324,-88.942660,UTC,10,Start
3,WT4,39.972739,-88.969221,UTC,10,Start
4,MAST1,40.042682,-89.058004,UTC,10,Start


In [8]:
run_custom_plots(scada_df=scada_df, assumed_rated_power_kw=assumed_rated_power_kw, rotor_diameter_m=rotor_diameter_m)

Custom plots saved to directory: C:\Users\snaylor\Documents\GitHub\wind-up\output\wedowind_example\custom_plots
Custom plots saved to directory: C:\Users\snaylor\Documents\GitHub\wind-up\output\wedowind_example\custom_plots


Amend dataframes based on reviewing the Custom Plots.

In [9]:
# based on the above I think the objects are MAST1, test=WT1 and ref=WT2
scada_df = scada_df.replace(
    {"TurbineName": {TurbineNames.TEST.value: "WT1", TurbineNames.REF.value: "WT2", "Mast": "MAST1"}}
)
# drop everything except the turbines from the metadata
metadata_df = metadata_df[metadata_df["Name"].isin(["WT1", "WT2"])]

### Construct Reanalysis

In [10]:
from wind_up.reanalysis_data import ReanalysisDataset

rng = np.random.default_rng(0)
rows = 100
reanalysis_dataset = ReanalysisDataset(
    id="dummy_reanalysis_data",
    data=pd.DataFrame(
        data={
            "100_m_hws_mean_mps": rng.uniform(5, 10, rows),
            "100_m_hwd_mean_deg-n_true": rng.uniform(0, 360, rows),
        },
        index=pd.DatetimeIndex(pd.date_range(start=scada_df.index.min(), periods=rows, freq="h", tz="UTC")),
    ),
)

reanalysis_dataset.data.head()

,100_m_hws_mean_mps,100_m_hwd_mean_deg-n_true
2010-07-30 22:40:00+00:00,8.184808,172.795653
2010-07-30 23:40:00+00:00,6.348934,83.654251
2010-07-31 00:40:00+00:00,5.204868,288.677008
2010-07-31 01:40:00+00:00,5.082638,332.470858
2010-07-31 02:40:00+00:00,9.066351,95.806898


# Construct `wind-up` Configuration

## Wind Farm Config

In [11]:
wtg_map = {
    x: {
        "name": x,
        "turbine_type": {
            "turbine_type": "unknown turbine type",
            "rotor_diameter_m": rotor_diameter_m,
            "rated_power_kw": assumed_rated_power_kw,
            "cutout_ws_mps": cutout_ws_mps,
            "normal_operation_pitch_range": (-10.0, 35.0),
            "normal_operation_genrpm_range": (0, 2000.0),
        },
    }
    for x in ["WT1", "WT2"]
}

cfg = WindUpConfig(
    assessment_name=ASSESSMENT_NAME,
    ref_wd_filter=[150, 240],  # apparent wake free sector
    use_lt_distribution=False,
    out_dir=OUTPUT_DIR / ASSESSMENT_NAME,
    test_wtgs=[wtg_map[x] for x in ["WT1"]],
    ref_wtgs=[wtg_map[x] for x in ["WT2"]],
    analysis_first_dt_utc_start=scada_df.index.min(),
    upgrade_first_dt_utc_start=scada_df[scada_df["upgrade status"] > 0].index.min(),
    analysis_last_dt_utc_start=scada_df[scada_df["upgrade status"] > 0].index.max(),
    years_offset_for_pre_period=1,
    lt_first_dt_utc_start=scada_df.index.min(),
    lt_last_dt_utc_start=scada_df.index.min()
    + (scada_df[scada_df["upgrade status"] > 0].index.max() - scada_df[scada_df["upgrade status"] > 0].index.min())
    - pd.Timedelta(minutes=10),
    detrend_first_dt_utc_start=scada_df.index.min(),
    detrend_last_dt_utc_start=scada_df[scada_df["upgrade status"] > 0].index.min()
    - pd.DateOffset(weeks=1)
    - pd.Timedelta(minutes=10),
    years_for_lt_distribution=1,
    years_for_detrend=1,
    ws_bin_width=1.0,
    asset={
        "name": "Mystery Wind Farm",
        "wtgs": list(wtg_map.values()),
    },
    missing_scada_data_fields=["YawAngleMin", "YawAngleMax"],
    prepost={
        "pre_first_dt_utc_start": scada_df.index.min(),
        "pre_last_dt_utc_start": scada_df.index.min()
        + (scada_df[scada_df["upgrade status"] > 0].index.max() - scada_df[scada_df["upgrade status"] > 0].index.min())
        - pd.Timedelta(minutes=10),
        "post_first_dt_utc_start": scada_df[scada_df["upgrade status"] > 0].index.min(),
        "post_last_dt_utc_start": scada_df[scada_df["upgrade status"] > 0].index.max(),
    },
    optimize_northing_corrections=False,
)

cfg

loaded WindUpConfig assessment_name: wedowind_example
loaded WindUpConfig assessment_name: wedowind_example
loaded WindUpConfig assessment_name: wedowind_example
pre analysis period (UTC): 2010-07-30 22:40 to 2010-09-29 19:20
pre analysis period (UTC): 2010-07-30 22:40 to 2010-09-29 19:20
pre analysis period (UTC): 2010-07-30 22:40 to 2010-09-29 19:20
post analysis period (UTC): 2011-04-25 21:50 to 2011-06-25 18:40
post analysis period (UTC): 2011-04-25 21:50 to 2011-06-25 18:40
post analysis period (UTC): 2011-04-25 21:50 to 2011-06-25 18:40
long term period (UTC): 2010-07-30 22:40 to 2010-09-29 19:20
long term period (UTC): 2010-07-30 22:40 to 2010-09-29 19:20
long term period (UTC): 2010-07-30 22:40 to 2010-09-29 19:20
detrend period (UTC): 2010-07-30 22:40 to 2011-04-18 21:50
detrend period (UTC): 2010-07-30 22:40 to 2011-04-18 21:50
detrend period (UTC): 2010-07-30 22:40 to 2011-04-18 21:50


WindUpConfig(assessment_name='wedowind_example', timebase_s=600, ignore_turbine_anemometer_data=False, require_test_wake_free=False, require_ref_wake_free=False, detrend_min_hours=24, ref_wd_filter=[150.0, 240.0], ref_hod_filter=None, filter_all_test_wtgs_together=False, use_lt_distribution=False, use_test_wtg_lt_distribution=True, out_dir=WindowsPath('C:/Users/snaylor/Documents/GitHub/wind-up/output/wedowind_example'), test_wtgs=[Turbine(name='WT1', turbine_type=TurbineType(turbine_type='unknown turbine type', rotor_diameter_m=80.0, rated_power_kw=1500.0, cutout_ws_mps=20.0, normal_operation_pitch_range=(-10.0, 35.0), normal_operation_genrpm_range=(0.0, 2000.0), rpm_v_pw_margin_factor=0.05, pitch_to_stall=False), latitude=nan, longitude=nan)], ref_wtgs=[Turbine(name='WT2', turbine_type=TurbineType(turbine_type='unknown turbine type', rotor_diameter_m=80.0, rated_power_kw=1500.0, cutout_ws_mps=20.0, normal_operation_pitch_range=(-10.0, 35.0), normal_operation_genrpm_range=(0.0, 2000.0)

## Plot Configuration

In [12]:
plot_cfg = PlotConfig(show_plots=False, save_plots=True, plots_dir=cfg.out_dir / "plots")

## Assessment Configs

In [13]:
assessment_inputs = AssessmentInputs.from_cfg(
    cfg=cfg,
    plot_cfg=plot_cfg,
    scada_df=scada_df[(scada_df["D"] < 70) | (scada_df["D"] > 150)],  # noqa PLR2004 filter out apparent mast waked sector
    metadata_df=metadata_df,
    reanalysis_datasets=[reanalysis_dataset],
    cache_dir=CACHE_DIR,
)

running wind_up analysis for wedowind_example
running wind_up analysis for wedowind_example
running wind_up analysis for wedowind_example
running load_smart_scada_and_md_from_file for 2010-07-30 22:40:00+00:00 to 2011-06-25 18:30:00+00:00
running load_smart_scada_and_md_from_file for 2010-07-30 22:40:00+00:00 to 2011-06-25 18:30:00+00:00
running load_smart_scada_and_md_from_file for 2010-07-30 22:40:00+00:00 to 2011-06-25 18:30:00+00:00
C:\Users\snaylor\Documents\GitHub\wind-up\wind_up\smart_data.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scada_raw["TurbineName"] = scada_raw["TurbineName"].astype("category")
loaded 2 turbines, 0.5 years per turbine
loaded 2 turbines, 0.5 years per turbine
loaded 2 turbines, 0.5 years per turbine
finishe

# Run Analysis

In [14]:
results_per_test_ref_df = run_wind_up_analysis(assessment_inputs)

test turbines: ['WT1']
test turbines: ['WT1']
test turbines: ['WT1']
test turbines: ['WT1']
ref list: ['WT2']
ref list: ['WT2']
ref list: ['WT2']
ref list: ['WT2']
turbines to test: ['WT1']
turbines to test: ['WT1']
turbines to test: ['WT1']
turbines to test: ['WT1']
could not calculate rolling windspeed diff
could not calculate rolling windspeed diff
could not calculate rolling windspeed diff
could not calculate rolling windspeed diff
C:\Users\snaylor\Documents\GitHub\wind-up\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\snaylor\Documents\GitHub\wind-up\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
WT1 check_for_ops_curve_shift warnings: abs(powercurve_shift) > 0.01: 0.072
WT1 check_for_ops_curve_shift warnings: abs(powercurve_shift) > 0.01: 0.072
WT1 check_for_

  0%|          | 0/400 [00:00<?, ?it/s]

block bootstrapping uncertainty analysis results (conf=90%):
  median = 2.7 %
  lower = 1.9 %
  upper = 3.8 %
  unc_one_sigma = 0.6 %
block bootstrapping uncertainty analysis results (conf=90%):
  median = 2.7 %
  lower = 1.9 %
  upper = 3.8 %
  unc_one_sigma = 0.6 %
block bootstrapping uncertainty analysis results (conf=90%):
  median = 2.7 %
  lower = 1.9 %
  upper = 3.8 %
  unc_one_sigma = 0.6 %
block bootstrapping uncertainty analysis results (conf=90%):
  median = 2.7 %
  lower = 1.9 %
  upper = 3.8 %
  unc_one_sigma = 0.6 %

cat A 1 sigma unc = 0.3 %

cat A 1 sigma unc = 0.3 %

cat A 1 sigma unc = 0.3 %

cat A 1 sigma unc = 0.3 %
abs reversal error / 2 = 0.1 %
abs reversal error / 2 = 0.1 %
abs reversal error / 2 = 0.1 %
abs reversal error / 2 = 0.1 %
bootstrap 1 sigma unc = 0.6 %
bootstrap 1 sigma unc = 0.6 %
bootstrap 1 sigma unc = 0.6 %
bootstrap 1 sigma unc = 0.6 %
missing bins scale factor = 1.000
missing bins scale factor = 1.000
missing bins scale factor = 1.000
missing bi

In [15]:
results_per_test_ref_df

,wind_up_version,time_calculated,preprocess_warning_counts,test_warning_counts,test_ref_warning_counts,test_wtg,test_pw_col,ref,ref_ws_col,uplift_frc,...,poweronly_uplift_frc,reversed_uplift_frc,reversal_error,lt_wtg_hours_raw,lt_wtg_hours_filt,test_max_ws_drift,test_max_ws_drift_pp_period,test_powercurve_shift,test_rpm_shift,test_pitch_shift
0,0.1.9,2024-09-10 14:56:31.754381+00:00,0,2,2,WT1,test_pw_clipped,WT2,ref_ws_est_blend,0.027311,...,0.023437,0.02571,0.002274,0,0,NaN,NaN,0.071621,0.0,0.0
